# Import Libraries 

In [ ]:
import cv2
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
from IPython.display import clear_output
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,KFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
cam = cv2.VideoCapture(0)
folder = "people/"+input('Person:').lower()

if not os.path.exists(folder):
    os.mkdir(folder)
    
    flag_start_capturing = False
    sample=1
   
    cv2.namedWindow("Face", cv2.WINDOW_NORMAL)

    while True:
        ret,frame = cam.read()
        gray = gray_scale(frame)
        faces_coord = detect_face(gray)

        if len(faces_coord):
            faces = normalize_faces(frame,faces_coord)
            cv2.imwrite(folder + '/' + str(sample)+'.jpg',faces[0])
            plot_show(faces[0],"Image saved:"+str(sample))
            clear_output(wait=True)
            if flag_start_capturing == True:
                sample += 1
            
        draw_rectangle(frame,faces_coord)
        cv2.imshow('Face',frame)
        keypress=cv2.waitKey(1)
        
        if keypress == ord('c'):
            
            if flag_start_capturing == False:
                flag_start_capturing = True
            
        
        if sample >500:
            break

    cam.release()
    cv2.destroyAllWindows()
else:
    print ("This name already exists.")

In [ ]:
def detect_face(frame):
   
        
    detector = cv2.CascadeClassifier("frontal_face.xml")

    faces = detector.detectMultiScale(frame,1.2,5)
    
    return faces

def gray_scale(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return image

def cut_faces(image, faces_coord):
    faces = []
      
    for (x, y, w, h) in faces_coord:
        
        faces.append(image[y: y + h, x : x + w ])
         
    return faces

def normalize_intensity(images):
    images_norm = []
    for image in images:
        images_norm.append(cv2.equalizeHist(image))
    return images_norm

def resize(images,size=(47,62)):
    image_resize = []
    
    for image in images:
        if image.shape < size:
            img_size = cv2.resize(image,size,interpolation=cv2.INTER_CUBIC)
        else:
            img_size = cv2.resize(image,size,interpolation=cv2.INTER_AREA)
        image_resize.append(img_size)
        
    return image_resize

def normalize_faces(frame, faces_coord):
    gray_frame = gray_scale(frame)
    faces = cut_faces(gray_frame, faces_coord)
    faces = normalize_intensity(faces)
    
    faces = resize(faces)
    return faces

def plot_show(image,title=""):
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    plt.axis("off")
    plt.title(title)
    plt.imshow(image,cmap="Greys_r")
    plt.show()

def draw_rectangle(image, coords):
    for (x, y, w, h) in coords:
        
        cv2.rectangle(image, (x , y), (x + w , y + h), (0,0,255),2)
        
def collect_dataset():
    images = []
    labels = []
    labels_dic = {}
   
    people = [person for person in os.listdir("people/")]
   
    for i, person in enumerate(people):
        labels_dic[i] = person
        for image in os.listdir("people/" + person):
            if image.endswith('.jpg'or 'jpeg'):
                images.append(cv2.imread("people/" + person + '/' + image, 0))
                labels.append(i)
    return (images, np.array(labels), labels_dic)

# Create dataset for unknown person here

In [ ]:
#path to lfw_home dataset of face images
basepath = 'C:\Users\MsWeather\Cdata\scikit_learn_data\lfw_home\lfw_funneled'
images = os.listdir(basepath) 
print (len(images))
data = images[:510]

for i,folder in enumerate(data,start=1):
    
    files=os.listdir(basepath+'\\'+folder)
    for k,img in enumerate(files,start=1):
        if img.endswith('.jpg'or'jpeg'):
            frame=cv2.imread(basepath+'\\'+folder+'\\'+img,0)
        faces_coord = detect_face(frame)
            if len(faces_coord):
                faces = cut_faces(frame, faces_coord)
                faces = normalize_intensity(faces)
                faces = resize(faces)
                cv2.imwrite('people/Unknown/' + str(i)+'.jpg',faces[0])
                
                break

In [ ]:
images, labels, labels_dic = collect_dataset()

In [ ]:
print (len(images))
print (labels_dic)

In [ ]:
X_train=np.asarray(images)

In [ ]:
train=X_train.reshape(len(X_train),-1)

In [ ]:
train.shape

In [ ]:
sc = StandardScaler()
X_train_sc = sc.fit_transform(train.astype(np.float64))
pca1 = PCA(n_components=.97)
new_train=pca1.fit_transform(X_train_sc)

In [ ]:
pca1.n_components_

In [ ]:
kf=KFold(n_splits=5,shuffle=True)
param_grid = {'C':[.0001,.001,.01,.1,1,10]}
gs_svc = GridSearchCV(SVC(kernel='linear',probability=True),param_grid=param_grid,cv=kf,scoring='accuracy')
gs_svc.fit(new_train,labels)
gs_svc.best_score_

In [ ]:
gs_svc.best_params_

In [ ]:
clf=gs_svc.best_estimator_
filename = 'svc_linear_face.pkl'
f=open(filename, 'wb')
pickle.dump(clf,f)
f.close()


In [ ]:
filename = 'svc_linear_face.pkl'
svc1 = pickle.load(open(filename, 'rb'))

In [ ]:
cam = cv2.VideoCapture(0)
font=cv2.FONT_HERSHEY_PLAIN
cv2.namedWindow("opencv_face", cv2.WINDOW_AUTOSIZE)
while True:
    ret,frame = cam.read()
    
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces_coord = detect_face(gray) # detect more than one face
    if len(faces_coord):
        faces = normalize_faces(frame, faces_coord)
        for i, face in enumerate(faces): # for each detected face
            
            t=face.reshape(1,-1)
            t=sc.transform(t.astype(np.float64))
            test = pca1.transform(t)    
            prob=svc1.predict_proba(test)
            confidence = svc1.decision_function(test)
            print (confidence)
            print (prob)
           
            pred = svc1.predict(test)
            print (pred,pred[0])
           
            name=labels_dic[pred[0]].capitalize()
            print (name)
           
            if prob[0][0]>.9:
                
                cv2.putText(frame, 'Hiten',(faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 2, (66, 53, 243), 2)
            
                
            elif prob[0][1]>.95:
                cv2.putText(frame,'kartikey',(faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 3, (66, 53, 243), 2)
            
            elif prob[0][2]>.95:
                cv2.putText(frame,'kuldeep',(faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 3, (66, 53, 243), 2)
            elif prob[0][3]>.85:
                cv2.putText(frame,'Mayank',(faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 3, (66, 53, 243), 2)
            elif prob[0][4]>.75:
                cv2.putText(frame,'Unknown',(faces_coord[i][0], faces_coord[i][1] - 10),
                            cv2.FONT_HERSHEY_PLAIN, 3, (66, 53, 243), 2)
             
        clear_output(wait = True)
        draw_rectangle(frame, faces_coord) # rectangle around face
        
    cv2.putText(frame, "ESC to exit", (5, frame.shape[0] - 5),cv2.FONT_HERSHEY_PLAIN, 1.3, (66, 53, 243), 2,cv2.LINE_AA)
    
    cv2.imshow("opencv_face", frame) # live feed in external
    if cv2.waitKey(5) == 27:
        
        break
        
cam.release()
cv2.destroyAllWindows()
        

In [ ]:
cam.release()